In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from dask.distributed import wait
import glob

SENSORS = ['acce','acce_uncali','gyro',
           'gyro_uncali','magn','magn_uncali','ahrs']

NFEAS = {
    'acce': 3,
    'acce_uncali': 3,
    'gyro': 3,
    'gyro_uncali': 3,
    'magn': 3,
    'magn_uncali': 3,
    'ahrs': 3,
    'wifi': 1,
    'ibeacon': 1,
    'waypoint': 3
}

ACOLS = ['timestamp','x','y','z']
        
FIELDS = {
    'acce': ACOLS,
    'acce_uncali': ACOLS,
    'gyro': ACOLS,
    'gyro_uncali': ACOLS,
    'magn': ACOLS,
    'magn_uncali': ACOLS,
    'ahrs': ACOLS,
    'wifi': ['timestamp','ssid','bssid','rssi','last_timestamp'],
    'ibeacon': ['timestamp','code','rssi','last_timestamp'],
    'waypoint': ['timestamp','x','y']
}

def to_frame(data, col):
    cols = FIELDS[col]
    is_dummy = False
    if data.shape[0]>0:
        df = pd.DataFrame(data, columns=cols)
    else:
        df = create_dummy_df(cols)
        is_dummy = True
    for col in df.columns:
        if 'timestamp' in col:
            df[col] = df[col].astype('int64')
    return df, is_dummy

def create_dummy_df(cols):
    df = pd.DataFrame()
    for col in cols:
        df[col] = [0]
        if col in ['ssid','bssid']:
            df[col] = df[col].map(str)
    return df

from dataclasses import dataclass

import numpy as np


@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            if len(line_data)>=5:
                ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            lastts = line_data[-1]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi, lastts]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)

    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

In [2]:
def get_test_dfs(PATH, test_files):
    dtest = get_test_df(PATH)
    buildings = set(dtest['building'].values.tolist())
    dws = {}
    ntest_files = []
    for fname in tqdm(test_files):
        path = fname.split('/')[-1].split('.')[0]
        mask = dtest['path'] == path
        dws[fname] = dtest.loc[mask, ['timestamp','x','y','floor','building','site_path_timestamp']].copy().reset_index(drop=True)
        ntest_files.append(fname)
    return dws

def get_test_df(PATH):
    dtest = pd.read_csv(f'{PATH}/sample_submission.csv')
    dtest['building'] = dtest['site_path_timestamp'].apply(lambda x: x.split('_')[0])
    dtest['path'] = dtest['site_path_timestamp'].apply(lambda x: x.split('_')[1])
    dtest['timestamp'] = dtest['site_path_timestamp'].apply(lambda x: x.split('_')[2])
    dtest['timestamp'] = dtest['timestamp'].astype('int64')
    dtest = dtest.sort_values(['path','timestamp']).reset_index(drop=True)
    return dtest

def get_time_gap(name):
    data = read_data_file(name)
    db,no_ibeacon = to_frame(data.ibeacon,'ibeacon')
#     print(db,no_ibeacon)
    
    if no_ibeacon==0:
        gap = db['last_timestamp'] - db['timestamp']
        assert gap.unique().shape[0]==1
        return gap.values[0],no_ibeacon
    
    if no_ibeacon==1:
        # Group wifis by timestamp
        wifi_groups = pd.DataFrame(data.wifi).groupby(0)   
        # Find which one is the most recent of all time points.
        est_ts = (wifi_groups[4].max().astype(int) - wifi_groups[0].max().astype(int)).max() 
        return est_ts,no_ibeacon

    

def fix_timestamp_test(df, gap):
    df['real_timestamp'] = df['timestamp'] + gap
    return df

In [3]:
test_files_ori = glob.glob('../input/indoor-location-navigation/test/*.txt')
test_files_ori[:4]

['../input/indoor-location-navigation/test/00ff0c9a71cc37a2ebdd0f05.txt',
 '../input/indoor-location-navigation/test/01c41f1aeba5c48c2c4dd568.txt',
 '../input/indoor-location-navigation/test/030b3d94de8acae7c936563d.txt',
 '../input/indoor-location-navigation/test/0389421238a7e2839701df0f.txt']

In [4]:
import dask
from dask.distributed import Client, wait, LocalCluster

# set n_workers to number of cores
client = Client(n_workers=8, 
                threads_per_worker=1)
client

/home/ec2-user/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:42097 Dashboard: http://127.0.0.1:39155/status,Cluster Workers: 8 Cores: 8 Memory: 32.89 GB


In [5]:
futures = []
for fname in tqdm(test_files_ori, total=len(test_files_ori)):
    f = client.submit(get_time_gap,fname)
    futures.append(f)
    
testpath2gap = {}
for f,fname in tqdm(zip(futures, test_files_ori), total=len(test_files_ori)):
    testpath2gap[fname.split('/')[-1].replace('.txt','')] = f.result()
    

100%|██████████| 626/626 [00:16<00:00, 37.39it/s] 


In [6]:
import pickle
with open('testpath2gap.pkl','wb') as f:
    pickle.dump(testpath2gap,f)